In [244]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.metrics import *
from sklearn.model_selection import KFold

from keras.layers import *
from keras import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras import backend as K

import tensorflow as tf

tqdm.pandas()
%matplotlib inline

## Loading Datasets

In [2]:
TRAIN_DF = pd.read_json('./train.json')
TEST_DF = pd.read_json('./test.json')

X_train = TRAIN_DF['audio_embedding'].tolist()
X_test = TEST_DF['audio_embedding'].tolist()
y_train = TRAIN_DF['is_turkey'].values

X_train = pad_sequences(X_train, maxlen=10) 
X_test = pad_sequences(X_test, maxlen=10) 

In [18]:
TRAIN_DF.head()

,audio_embedding,end_time_seconds_youtube_clip,is_turkey,start_time_seconds_youtube_clip,vid_id
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",70,0,60,kDCk3hLIVXo
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",40,1,30,DPcGzqHoo7Y
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",240,1,230,7yM63MTHh5k
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",520,1,510,luG3RmUAxxM
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",10,0,0,PIm3cjxTpOk


In [19]:
TEST_DF.head()

,audio_embedding,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip,vid_id
0,"[[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...",10,0,pyKh38FXD3E
1,"[[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...",40,30,THhP1idrWXA
2,"[[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...",40,30,jsw3T6GY2Nw
3,"[[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...",24,14,nFkXTMHcjMU
4,"[[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...",40,30,Au8g9kAlrLQ


## Our Deep Learning Model Baseline 

In [3]:
# https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [4]:
import tensorflow as tf
# define roc_callback, inspired by https://github.com/keras-team/keras/issues/6050#issuecomment-329996505
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(y_true, y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [5]:
def get_model(learn_rate=0.01, momentum=0.8, dropout_rate1=0.1, dropout_rate2=0.0, neurons=128):
    model = Sequential()
    model.add(BatchNormalization(momentum=momentum,input_shape=(10, 128)))
    model.add(Bidirectional(GRU(neurons, dropout=dropout_rate1, recurrent_dropout=dropout_rate2, activation='relu', return_sequences=True)))
    model.add(Attention(10))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learn_rate), metrics=[auc_roc])
    return model

In [8]:
def train_model(X_train, y_train, X_test, n_splits=5, verbose=2):

    missfits = [] ## missclassified items
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=424242)
    preds = []
    historys = []
    fold = 0
    aucs = 0.0
    threshold = 0.5

    for train_idx, val_idx in kf.split(X_train):
        x_train_f = X_train[train_idx]
        y_train_f = y_train[train_idx]
        x_val_f = X_train[val_idx]
        y_val_f = y_train[val_idx]

        model = get_model(neurons=128, momentum=0.6, learn_rate=0.001, dropout_rate2=0.1, dropout_rate1=0.1)
        history = model.fit(x_train_f, y_train_f,
            batch_size=128,
            epochs=20,
            verbose = verbose,
            validation_data=(x_val_f, y_val_f),
            callbacks = [])
        
        preds_val = model.predict([x_val_f], batch_size=512)

        
        preds_test = model.predict(X_test)
        preds.append(preds_test)
        
        historys.append(history)
        
        score = roc_auc_score(y_val_f, preds_val)
        aucs += score
    
        preds_val = preds_val > threshold
        for idx in range(len(x_val_f)):
            if preds_val[0][0] != y_val_f[idx]:
                missfits.append(TRAIN_DF.values[train_idx[idx]])
    
        fold+=1
    
        print('Fold {}, AUC = {}'.format(fold, score))
    
    preds = np.asarray(preds)[...,0]
    preds = np.mean(preds, axis=0)
    print("Cross Validation AUC = {}".format(aucs/n_splits))
    
    return (preds, historys, missfits)

    

    
    
    
    

    
    

In [11]:
preds, historys, missfits = train_model(X_train, y_train, X_test, n_splits=5, verbose=0)

Fold 1, AUC = 0.9855699855699855
Fold 2, AUC = 0.9850862312694374
Fold 3, AUC = 0.9835907335907337
Fold 4, AUC = 0.9936837519965152
Fold 5, AUC = 0.9913315850815851
Cross Validation AUC = 0.9878524575016513


In [13]:
?pd.DataFrame

## Getting more data from external sources

In [15]:
! mkdir -p html/train/0
! mkdir -p html/train/1
! mkdir -p html/test

In [14]:
BASE_URL = "https://www.youtube.com/watch?v="

In [30]:
vid_ids_train_0 = TRAIN_DF[TRAIN_DF.is_turkey==0]['vid_id'].values
vid_ids_train_1 = TRAIN_DF[TRAIN_DF.is_turkey==1]['vid_id'].values
vid_ids_test = TEST_DF['vid_id'].values

In [31]:
print(len(vid_ids_train_0))
print(len(vid_ids_train_1))
print(len(vid_ids_test))

704
491
1196


In [42]:
prefix = "./html/train/0/"
for vid_id in tqdm(vid_ids_train_0):
    url = BASE_URL + vid_id
    output_file = prefix + vid_id + ".html"
    !wget '{url}' -O '{output_file}' -nv



  0%|          | 0/704 [00:00<?, ?it/s]

2018-11-25 11:40:01 URL:https://www.youtube.com/watch?v=kDCk3hLIVXo [148732] -> "./html/train/0/kDCk3hLIVXo.html" [1]




  0%|          | 1/704 [00:00<05:10,  2.26it/s]

2018-11-25 11:40:01 URL:https://www.youtube.com/watch?v=PIm3cjxTpOk [199692] -> "./html/train/0/PIm3cjxTpOk.html" [1]




  0%|          | 2/704 [00:00<05:26,  2.15it/s]

...



100%|█████████▉| 702/704 [06:18<00:01,  1.92it/s]

2018-11-25 11:46:19 URL:https://www.youtube.com/watch?v=opqJ1SZmyGk [211864] -> "./html/train/0/opqJ1SZmyGk.html" [1]




100%|█████████▉| 703/704 [06:18<00:00,  1.72it/s]

2018-11-25 11:46:20 URL:https://www.youtube.com/watch?v=aFHQY8eX5LI [211765] -> "./html/train/0/aFHQY8eX5LI.html" [1]




100%|██████████| 704/704 [06:19<00:00,  1.68it/s]



In [43]:
prefix = "./html/train/1/"
for vid_id in tqdm(vid_ids_train_1):
    url = BASE_URL + vid_id
    output_file = prefix + vid_id + ".html"
    !wget '{url}' -O '{output_file}' -nv



  0%|          | 0/491 [00:00<?, ?it/s]

2018-11-25 11:46:20 URL:https://www.youtube.com/watch?v=DPcGzqHoo7Y [203153] -> "./html/train/1/DPcGzqHoo7Y.html" [1]




  0%|          | 1/491 [00:00<04:22,  1.86it/s]

2018-11-25 11:46:21 URL:https://www.youtube.com/watch?v=7yM63MTHh5k [180463] -> "./html/train/1/7yM63MTHh5k.html" [1]




  0%|          | 2/491 [00:01<04:21,  1.87it/s]

...



100%|█████████▉| 489/491 [04:24<00:01,  1.86it/s]

2018-11-25 11:50:45 URL:https://www.youtube.com/watch?v=2TK9q8qFqVE [196768] -> "./html/train/1/2TK9q8qFqVE.html" [1]




100%|█████████▉| 490/491 [04:25<00:00,  1.90it/s]

2018-11-25 11:50:45 URL:https://www.youtube.com/watch?v=DbJJoQfo3kc [224983] -> "./html/train/1/DbJJoQfo3kc.html" [1]




100%|██████████| 491/491 [04:25<00:00,  1.93it/s]



In [44]:
prefix = "./html/test/"
for vid_id in tqdm(vid_ids_test):
    url = BASE_URL + vid_id
    output_file = prefix + vid_id + ".html"
    !wget '{url}' -O '{output_file}' -nv



  0%|          | 0/1196 [00:00<?, ?it/s]

2018-11-25 11:50:46 URL:https://www.youtube.com/watch?v=pyKh38FXD3E [222873] -> "./html/test/pyKh38FXD3E.html" [1]




  0%|          | 1/1196 [00:00<10:02,  1.98it/s]

2018-11-25 11:50:46 URL:https://www.youtube.com/watch?v=THhP1idrWXA [214930] -> "./html/test/THhP1idrWXA.html" [1]




  0%|          | 2/1196 [00:01<10:28,  1.90it/s]

...



100%|█████████▉| 1194/1196 [11:20<00:01,  1.83it/s]

2018-11-25 12:02:06 URL:https://www.youtube.com/watch?v=f6OhmrIdB-g [167339] -> "./html/test/f6OhmrIdB-g.html" [1]




100%|█████████▉| 1195/1196 [11:21<00:00,  1.81it/s]

2018-11-25 12:02:07 URL:https://www.youtube.com/watch?v=vCxGJelmAeg [205723] -> "./html/test/vCxGJelmAeg.html" [1]




100%|██████████| 1196/1196 [11:21<00:00,  1.77it/s]



## Generate More Features

In [239]:
import math
from collections import Counter
from bs4 import BeautifulSoup
from glob import glob
from textblob import TextBlob as tb

import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words


def stem(words):
    stems = stem_words(words)
    return stems


def clean_html(soup):
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    # get text
    text = soup.get_text()
    
    return text.lower()

In [329]:
global VIDID_TO_FEATURES, FEATURES_LIST
VIDID_TO_FEATURES = dict()
FEATURES_LIST = [
    "turkey_in_title",
    "hunt_in_title",
    "wild_in_title",
    "laugh_in_title",
    "bearded_in_title",
    "bow_in_title",
    "gobbler_in_title",
    "guajolote_in_title",
    "pavo_in_title",
    "season_in_title",
    "chuckle_in_title",
    "giggle_in_title",
    "tickle_in_title",
    
     "game_in_title",
    "rire_in_title",
    "fou_in_title",
    "funny_in_title",
    "turkeys_in_title",
    "animal_in_title",
    "fun_in_title",
    "laughter_in_title",
    "contagious_in_title",
    "gobble_in_title",
    "dinde_in_title",
    'counts_game_zero',
    'counts_game',
    'counts_rire',
    'counts_fou',
    'counts_funny',
    'counts_turkeys',
    'counts_animal',
    'counts_fun',
    'counts_laughter',
    'counts_contagious',
    'counts_gobble',
    'counts_dinde',
    "counts_turkey",
    "counts_wild",
    "counts_hunt",
    "counts_laugh",
    "counts_bearded",
    "counts_bow",
    "counts_gobbler",
    "counts_guajolote",
    "counts_pavo",
    "counts_season",
    "counts_chuckle",
    "counts_giggle",
    "counts_tickle",  
]

In [330]:
def generate_features_from_html(): 
    
    html_files = glob("./html/**/*.html", recursive=True)
    for html_file in tqdm(html_files):
        vid_id = html_file.split('/')[-1].split('.')[0]

        f = open(html_file)     
        soup = BeautifulSoup(f, "html5lib")
        
        title = " ".join( [x.text.strip().lower() for x in soup.find_all('h1')] )
        
        # IN title Features 
        turkey_in_title = "turkey" in title
        hunt_in_title = "hunt" in title
        wild_in_title = "wild" in title
        laugh_in_title = "laugh" in title
        bearded_in_title = "bearded" in title
        bow_in_title = "bow" in title
        gobbler_in_title = "gobbler" in title
        guajolote_in_title = "guajolote" in title
        pavo_in_title = "pavo" in title
        season_in_title = "season" in title    
        chuckle_in_title = "chuckle" in title
        giggle_in_title = "giggle" in title
        tickle_in_title = "tickle" in title
        
        game_in_title = "game" in title
        rire_in_title = "rire" in title
        fou_in_title = "fou" in title
        funny_in_title = "funny" in title  
        turkeys_in_title = "turkeys" in title 
        animal_in_title = "animal" in title 
        fun_in_title = "fun" in title 
        laughter_in_title = "laughter" in title 
        contagious_in_title = "contagious" in title 
        gobble_in_title = "gobble" in title
        dinde_in_title = "dinde" in title
        
        # Word Count Features
        from collections import Counter
        data = clean_html(soup)
        words = re.findall(r'\w+', data)
        counts = Counter(words)
        
        counts_turkey = counts['turkey']
        counts_wild = counts['wild']
        counts_hunt = counts['hunt']
        counts_laugh = counts['laugh']
        counts_bearded = counts["bearded"]
        counts_bow = counts["bow"]
        counts_gobbler = counts["gobbler"]
        counts_guajolote = counts["guajolote"]
        counts_pavo = counts["pavo"]
        counts_season = counts["season"] 
        counts_chuckle = counts["chuckle"]
        counts_giggle = counts["giggle"]
        counts_tickle = counts["tickle"]
        
        counts_game = counts["game"]
        counts_rire = counts["rire"]
        counts_fou = counts["fou"]
        counts_funny = counts["funny"]
        counts_turkeys = counts["turkeys"]
        counts_animal = counts["animal"]
        counts_fun = counts["fun"]
        counts_laughter = counts["laughter"]
        counts_contagious = counts["contagious"] 
        counts_gobble = counts["gobble"]
        counts_dinde = counts["dinde"]
        
        VIDID_TO_FEATURES[vid_id] = {
            "turkey_in_title": turkey_in_title,
            "hunt_in_title": hunt_in_title,
            "wild_in_title": wild_in_title,
            "laugh_in_title": laugh_in_title,
            "bearded_in_title": bearded_in_title,
            "bow_in_title":  bow_in_title,
            "gobbler_in_title":  gobbler_in_title,
            "guajolote_in_title": guajolote_in_title,
            "pavo_in_title": pavo_in_title,
            "season_in_title": season_in_title,
            "chuckle_in_title": chuckle_in_title,
            "giggle_in_title": giggle_in_title,
            "tickle_in_title": tickle_in_title,
            "game_in_title": game_in_title,
            "rire_in_title": rire_in_title,
            "fou_in_title": fou_in_title,
            "funny_in_title": funny_in_title,
            "turkeys_in_title": turkeys_in_title,
            "animal_in_title": animal_in_title,
            "fun_in_title": fun_in_title,
            "laughter_in_title": laughter_in_title,
            "contagious_in_title": contagious_in_title,
            "gobble_in_title": gobble_in_title,
            "dinde_in_title": dinde_in_title,
            "counts_turkey": counts_turkey,
            "counts_wild": counts_wild,
            "counts_hunt": counts_hunt,
            "counts_laugh": counts_laugh,
            "counts_bearded": counts_bearded,
            "counts_bow": counts_bow,
            "counts_gobbler": counts_gobbler,
            "counts_guajolote": counts_guajolote,
            "counts_pavo": counts_pavo,
            "counts_season": counts_season,
            "counts_chuckle": counts_chuckle,
            "counts_giggle": counts_giggle,
            "counts_tickle": counts_tickle,
            "counts_game": counts_game,
            "counts_rire": counts_rire,
            "counts_fou": counts_fou,
            "counts_funny": counts_funny,
            "counts_turkeys": counts_turkeys,
            "counts_animal": counts_animal,
            "counts_fun": counts_fun,
            "counts_laughter": counts_laughter,
            "counts_contagious": counts_contagious,
            "counts_gobble": counts_gobble,
            "counts_dinde": counts_dinde,
        }
        
        f.close()
   

In [332]:
generate_features_from_html()



  0%|          | 0/2391 [00:00<?, ?it/s]

  0%|          | 2/2391 [00:00<04:37,  8.61it/s]

  0%|          | 3/2391 [00:00<04:57,  8.02it/s]

  0%|          | 4/2391 [00:00<05:14,  7.58it/s]

  0%|          | 5/2391 [00:00<05:28,  7.27it/s]

  0%|          | 6/2391 [00:00<05:34,  7.12it/s]

  0%|          | 7/2391 [00:00<05:37,  7.07it/s]

  0%|          | 8/2391 [00:01<05:39,  7.01it/s]

  0%|          | 9/2391 [00:01<05:39,  7.01it/s]

  0%|          | 10/2391 [00:01<05:42,  6.94it/s]

  0%|          | 11/2391 [00:01<05:47,  6.85it/s]

  1%|          | 12/2391 [00:01<05:49,  6.80it/s]

  1%|          | 13/2391 [00:01<05:49,  6.81it/s]

  1%|          | 14/2391 [00:01<05:51,  6.76it/s]

  1%|          | 15/2391 [00:02<05:51,  6.76it/s]

  1%|          | 16/2391 [00:02<05:53,  6.73it/s]

  1%|          | 17/2391 [00:02<05:58,  6.61it/s]

  1%|          | 18/2391 [00:02<05:53,  6.71it/s]

  1%|          | 19/2391 [00:02<05:52,  6.73it/s]

  1%|          | 20/2391 [00:02<05:56,  6.66it

  6%|▌         | 148/2391 [00:21<05:31,  6.77it/s]

  6%|▌         | 149/2391 [00:21<05:30,  6.78it/s]

  6%|▋         | 150/2391 [00:21<05:28,  6.81it/s]

  6%|▋         | 151/2391 [00:21<05:29,  6.81it/s]

  6%|▋         | 152/2391 [00:22<05:30,  6.77it/s]

  6%|▋         | 153/2391 [00:22<05:32,  6.73it/s]

  6%|▋         | 154/2391 [00:22<05:27,  6.84it/s]

  6%|▋         | 155/2391 [00:22<05:28,  6.80it/s]

  7%|▋         | 156/2391 [00:22<05:31,  6.75it/s]

  7%|▋         | 157/2391 [00:22<05:31,  6.75it/s]

  7%|▋         | 158/2391 [00:23<05:37,  6.62it/s]

  7%|▋         | 159/2391 [00:23<05:36,  6.64it/s]

  7%|▋         | 160/2391 [00:23<05:38,  6.59it/s]

  7%|▋         | 161/2391 [00:23<05:31,  6.73it/s]

  7%|▋         | 162/2391 [00:23<05:29,  6.77it/s]

  7%|▋         | 163/2391 [00:23<05:28,  6.77it/s]

  7%|▋         | 164/2391 [00:23<05:31,  6.72it/s]

  7%|▋         | 165/2391 [00:24<05:28,  6.78it/s]

  7%|▋         | 166/2391 [00:24<05:35,  6.64it/s]

  7%|▋      

 12%|█▏        | 292/2391 [00:42<04:53,  7.16it/s]

 12%|█▏        | 293/2391 [00:42<04:52,  7.18it/s]

 12%|█▏        | 294/2391 [00:42<04:57,  7.06it/s]

 12%|█▏        | 295/2391 [00:42<04:55,  7.09it/s]

 12%|█▏        | 296/2391 [00:42<04:54,  7.11it/s]

 12%|█▏        | 297/2391 [00:42<04:53,  7.13it/s]

 12%|█▏        | 298/2391 [00:43<04:52,  7.15it/s]

 13%|█▎        | 299/2391 [00:43<05:00,  6.96it/s]

 13%|█▎        | 300/2391 [00:43<05:01,  6.94it/s]

 13%|█▎        | 301/2391 [00:43<05:01,  6.92it/s]

 13%|█▎        | 302/2391 [00:43<05:03,  6.89it/s]

 13%|█▎        | 303/2391 [00:43<05:00,  6.96it/s]

 13%|█▎        | 304/2391 [00:43<04:59,  6.98it/s]

 13%|█▎        | 305/2391 [00:44<04:59,  6.97it/s]

 13%|█▎        | 306/2391 [00:44<05:00,  6.93it/s]

 13%|█▎        | 307/2391 [00:44<04:57,  7.00it/s]

 13%|█▎        | 308/2391 [00:44<04:57,  7.00it/s]

 13%|█▎        | 309/2391 [00:44<04:59,  6.95it/s]

 13%|█▎        | 311/2391 [00:44<04:27,  7.78it/s]

 13%|█▎     

 18%|█▊        | 436/2391 [01:09<04:44,  6.86it/s]

 18%|█▊        | 437/2391 [01:10<04:43,  6.90it/s]

 18%|█▊        | 438/2391 [01:10<04:44,  6.86it/s]

 18%|█▊        | 439/2391 [01:10<04:45,  6.83it/s]

 18%|█▊        | 440/2391 [01:10<04:44,  6.87it/s]

 18%|█▊        | 441/2391 [01:10<04:44,  6.85it/s]

 18%|█▊        | 442/2391 [01:10<04:30,  7.20it/s]

 19%|█▊        | 443/2391 [01:10<04:36,  7.04it/s]

 19%|█▊        | 444/2391 [01:11<04:40,  6.95it/s]

 19%|█▊        | 445/2391 [01:11<04:34,  7.10it/s]

 19%|█▊        | 446/2391 [01:11<04:35,  7.07it/s]

 19%|█▊        | 447/2391 [01:11<04:32,  7.12it/s]

 19%|█▊        | 448/2391 [01:11<04:34,  7.07it/s]

 19%|█▉        | 449/2391 [01:11<04:40,  6.93it/s]

 19%|█▉        | 450/2391 [01:11<04:38,  6.97it/s]

 19%|█▉        | 451/2391 [01:12<04:37,  6.99it/s]

 19%|█▉        | 452/2391 [01:12<04:37,  7.00it/s]

 19%|█▉        | 453/2391 [01:12<04:30,  7.16it/s]

 19%|█▉        | 454/2391 [01:12<04:35,  7.03it/s]

 19%|█▉     

 24%|██▍       | 578/2391 [01:30<04:32,  6.67it/s]

 24%|██▍       | 579/2391 [01:30<04:20,  6.96it/s]

 24%|██▍       | 580/2391 [01:30<04:22,  6.91it/s]

 24%|██▍       | 581/2391 [01:31<04:21,  6.93it/s]

 24%|██▍       | 582/2391 [01:31<04:23,  6.87it/s]

 24%|██▍       | 583/2391 [01:31<04:24,  6.82it/s]

 24%|██▍       | 584/2391 [01:31<04:23,  6.85it/s]

 24%|██▍       | 585/2391 [01:31<04:23,  6.85it/s]

 25%|██▍       | 586/2391 [01:31<04:21,  6.89it/s]

 25%|██▍       | 587/2391 [01:31<04:23,  6.86it/s]

 25%|██▍       | 588/2391 [01:32<04:24,  6.82it/s]

 25%|██▍       | 589/2391 [01:32<04:26,  6.75it/s]

 25%|██▍       | 590/2391 [01:32<04:31,  6.64it/s]

 25%|██▍       | 591/2391 [01:32<04:28,  6.70it/s]

 25%|██▍       | 592/2391 [01:32<04:24,  6.80it/s]

 25%|██▍       | 593/2391 [01:32<04:23,  6.82it/s]

 25%|██▍       | 594/2391 [01:32<04:12,  7.13it/s]

 25%|██▍       | 595/2391 [01:33<04:11,  7.15it/s]

 25%|██▍       | 596/2391 [01:33<04:12,  7.11it/s]

 25%|██▍    

...

 72%|███████▏  | 1716/2391 [04:25<01:34,  7.15it/s]

 72%|███████▏  | 1717/2391 [04:25<01:35,  7.06it/s]

 72%|███████▏  | 1718/2391 [04:25<01:34,  7.10it/s]

 72%|███████▏  | 1719/2391 [04:26<01:34,  7.11it/s]

 72%|███████▏  | 1720/2391 [04:26<01:33,  7.15it/s]

 72%|███████▏  | 1721/2391 [04:26<01:33,  7.18it/s]

 72%|███████▏  | 1722/2391 [04:26<01:33,  7.13it/s]

 72%|███████▏  | 1723/2391 [04:26<01:33,  7.15it/s]

 72%|███████▏  | 1724/2391 [04:26<01:34,  7.04it/s]

 72%|███████▏  | 1725/2391 [04:26<01:34,  7.06it/s]

 72%|███████▏  | 1726/2391 [04:27<01:33,  7.13it/s]

 72%|███████▏  | 1727/2391 [04:27<01:31,  7.22it/s]

 72%|███████▏  | 1728/2391 [04:27<01:32,  7.18it/s]

 72%|███████▏  | 1729/2391 [04:27<01:31,  7.21it/s]

 72%|███████▏  | 1730/2391 [04:27<01:31,  7.25it/s]

 72%|███████▏  | 1731/2391 [04:27<01:31,  7.21it/s]

 72%|███████▏  | 1732/2391 [04:27<01:30,  7.27it/s]

 72%|███████▏  | 1733/2391 [04:28<01:29,  7.37it/s]

 73%|███████▎  | 1734/2391 [04:28<01:29,  7.36

 78%|███████▊  | 1857/2391 [04:44<01:13,  7.26it/s]

 78%|███████▊  | 1858/2391 [04:45<01:14,  7.18it/s]

 78%|███████▊  | 1859/2391 [04:45<01:14,  7.13it/s]

 78%|███████▊  | 1860/2391 [04:45<01:13,  7.19it/s]

 78%|███████▊  | 1861/2391 [04:45<01:13,  7.22it/s]

 78%|███████▊  | 1862/2391 [04:45<01:12,  7.26it/s]

 78%|███████▊  | 1863/2391 [04:45<01:12,  7.25it/s]

 78%|███████▊  | 1864/2391 [04:45<01:11,  7.37it/s]

 78%|███████▊  | 1866/2391 [04:46<01:03,  8.26it/s]

 78%|███████▊  | 1867/2391 [04:46<01:05,  7.94it/s]

 78%|███████▊  | 1868/2391 [04:46<01:08,  7.60it/s]

 78%|███████▊  | 1869/2391 [04:46<01:10,  7.42it/s]

 78%|███████▊  | 1870/2391 [04:46<01:07,  7.75it/s]

 78%|███████▊  | 1871/2391 [04:46<01:08,  7.54it/s]

 78%|███████▊  | 1872/2391 [04:46<01:09,  7.48it/s]

 78%|███████▊  | 1873/2391 [04:46<01:10,  7.37it/s]

 78%|███████▊  | 1874/2391 [04:47<01:08,  7.53it/s]

 78%|███████▊  | 1875/2391 [04:47<01:10,  7.29it/s]

 78%|███████▊  | 1876/2391 [04:47<01:10,  7.27

 84%|████████▎ | 1997/2391 [05:11<00:56,  7.02it/s]

 84%|████████▎ | 1998/2391 [05:11<00:55,  7.06it/s]

 84%|████████▎ | 1999/2391 [05:12<00:54,  7.19it/s]

 84%|████████▎ | 2000/2391 [05:12<00:54,  7.13it/s]

 84%|████████▎ | 2001/2391 [05:12<00:55,  7.06it/s]

 84%|████████▎ | 2002/2391 [05:12<00:54,  7.08it/s]

 84%|████████▍ | 2003/2391 [05:12<00:53,  7.22it/s]

 84%|████████▍ | 2004/2391 [05:12<00:53,  7.23it/s]

 84%|████████▍ | 2005/2391 [05:12<00:53,  7.27it/s]

 84%|████████▍ | 2006/2391 [05:13<00:52,  7.32it/s]

 84%|████████▍ | 2007/2391 [05:13<00:52,  7.32it/s]

 84%|████████▍ | 2008/2391 [05:13<00:53,  7.20it/s]

 84%|████████▍ | 2009/2391 [05:13<00:53,  7.09it/s]

 84%|████████▍ | 2010/2391 [05:13<00:52,  7.24it/s]

 84%|████████▍ | 2011/2391 [05:13<00:52,  7.21it/s]

 84%|████████▍ | 2012/2391 [05:13<00:53,  7.08it/s]

 84%|████████▍ | 2013/2391 [05:14<00:52,  7.17it/s]

 84%|████████▍ | 2014/2391 [05:14<00:52,  7.20it/s]

 84%|████████▍ | 2015/2391 [05:14<00:51,  7.27

 89%|████████▉ | 2136/2391 [05:31<00:35,  7.15it/s]

 89%|████████▉ | 2137/2391 [05:31<00:36,  7.00it/s]

 89%|████████▉ | 2138/2391 [05:31<00:36,  6.96it/s]

 89%|████████▉ | 2139/2391 [05:31<00:36,  6.92it/s]

 90%|████████▉ | 2140/2391 [05:31<00:35,  7.05it/s]

 90%|████████▉ | 2141/2391 [05:32<00:35,  7.07it/s]

 90%|████████▉ | 2142/2391 [05:32<00:35,  7.05it/s]

 90%|████████▉ | 2143/2391 [05:32<00:35,  6.95it/s]

 90%|████████▉ | 2144/2391 [05:32<00:35,  6.94it/s]

 90%|████████▉ | 2145/2391 [05:32<00:35,  6.83it/s]

 90%|████████▉ | 2146/2391 [05:32<00:35,  6.89it/s]

 90%|████████▉ | 2147/2391 [05:32<00:35,  6.91it/s]

 90%|████████▉ | 2148/2391 [05:33<00:35,  6.83it/s]

 90%|████████▉ | 2149/2391 [05:33<00:35,  6.79it/s]

 90%|████████▉ | 2150/2391 [05:33<00:34,  6.92it/s]

 90%|████████▉ | 2151/2391 [05:33<00:34,  6.93it/s]

 90%|█████████ | 2152/2391 [05:33<00:35,  6.80it/s]

 90%|█████████ | 2153/2391 [05:33<00:34,  6.91it/s]

 90%|█████████ | 2154/2391 [05:33<00:33,  6.99

 95%|█████████▌| 2274/2391 [05:50<00:16,  7.20it/s]

 95%|█████████▌| 2275/2391 [05:50<00:15,  7.30it/s]

 95%|█████████▌| 2276/2391 [05:50<00:15,  7.44it/s]

 95%|█████████▌| 2277/2391 [05:50<00:15,  7.44it/s]

 95%|█████████▌| 2278/2391 [05:51<00:15,  7.43it/s]

 95%|█████████▌| 2279/2391 [05:51<00:14,  7.47it/s]

 95%|█████████▌| 2280/2391 [05:51<00:14,  7.51it/s]

 95%|█████████▌| 2281/2391 [05:51<00:14,  7.38it/s]

 95%|█████████▌| 2282/2391 [05:51<00:14,  7.48it/s]

 95%|█████████▌| 2283/2391 [05:51<00:14,  7.44it/s]

 96%|█████████▌| 2284/2391 [05:51<00:14,  7.51it/s]

 96%|█████████▌| 2285/2391 [05:52<00:13,  7.65it/s]

 96%|█████████▌| 2286/2391 [05:52<00:13,  7.69it/s]

 96%|█████████▌| 2287/2391 [05:52<00:13,  7.65it/s]

 96%|█████████▌| 2288/2391 [05:52<00:13,  7.52it/s]

 96%|█████████▌| 2289/2391 [05:52<00:13,  7.51it/s]

 96%|█████████▌| 2290/2391 [05:52<00:13,  7.58it/s]

 96%|█████████▌| 2291/2391 [05:52<00:13,  7.56it/s]

 96%|█████████▌| 2292/2391 [05:52<00:12,  7.64

In [333]:
l_turkey_in_title = []
l_hunt_in_title = []
l_wild_in_title = []
l_laugh_in_title = []
l_bearded_in_title = []
l_bow_in_title = []
l_gobbler_in_title = []
l_guajolote_in_title = []
l_pavo_in_title = []
l_season_in_title = []
l_chuckle_in_title = []
l_giggle_in_title = []
l_tickle_in_title = []
l_counts_turkey = []
l_counts_wild = []
l_counts_hunt = []
l_counts_laugh = []
l_counts_bearded = []
l_counts_bow = []
l_counts_gobbler = []
l_counts_guajolote = []
l_counts_pavo = []
l_counts_season = []
l_counts_chuckle = []
l_counts_giggle = []
l_counts_tickle = []

l_counts_game = []
l_counts_rire = []
l_counts_fou = []
l_counts_funny = []
l_counts_turkeys = []
l_counts_animal = []
l_counts_fun = []
l_counts_laughter = []
l_counts_contagious = []
l_counts_gobble = []
l_counts_dinde = []

l_game_in_title = []
l_rire_in_title = []
l_fou_in_title = []
l_funny_in_title = [] 
l_turkeys_in_title = []
l_animal_in_title = []
l_fun_in_title = []
l_laughter_in_title = []
l_contagious_in_title = []
l_gobble_in_title = []
l_dinde_in_title = []

for vid_id in tqdm(TRAIN_DF.vid_id.values):
 
    l_turkey_in_title.append(VIDID_TO_FEATURES[vid_id]["turkey_in_title"])
    l_hunt_in_title.append(VIDID_TO_FEATURES[vid_id]["hunt_in_title"])
    l_wild_in_title.append(VIDID_TO_FEATURES[vid_id]["wild_in_title"])
    l_laugh_in_title.append(VIDID_TO_FEATURES[vid_id]["laugh_in_title"])
    l_bearded_in_title.append(VIDID_TO_FEATURES[vid_id]["bearded_in_title"])
    l_bow_in_title.append(VIDID_TO_FEATURES[vid_id]["bow_in_title"])
    l_gobbler_in_title.append(VIDID_TO_FEATURES[vid_id]["gobbler_in_title"])
    l_guajolote_in_title.append(VIDID_TO_FEATURES[vid_id]["guajolote_in_title"])
    l_pavo_in_title.append(VIDID_TO_FEATURES[vid_id]["pavo_in_title"])
    l_season_in_title.append(VIDID_TO_FEATURES[vid_id]["season_in_title"])
    l_chuckle_in_title.append(VIDID_TO_FEATURES[vid_id]["chuckle_in_title"])
    l_giggle_in_title.append(VIDID_TO_FEATURES[vid_id]["giggle_in_title"])
    l_tickle_in_title.append(VIDID_TO_FEATURES[vid_id]["tickle_in_title"])
    
    l_game_in_title.append(VIDID_TO_FEATURES[vid_id]["game_in_title"])
    l_rire_in_title.append(VIDID_TO_FEATURES[vid_id]["rire_in_title"])
    l_fou_in_title.append(VIDID_TO_FEATURES[vid_id]["fou_in_title"])
    l_funny_in_title.append(VIDID_TO_FEATURES[vid_id]["funny_in_title"])
    l_turkeys_in_title.append(VIDID_TO_FEATURES[vid_id]["turkeys_in_title"])
    l_animal_in_title.append(VIDID_TO_FEATURES[vid_id]["animal_in_title"])
    l_fun_in_title.append(VIDID_TO_FEATURES[vid_id]["fun_in_title"])
    l_laughter_in_title.append(VIDID_TO_FEATURES[vid_id]["laughter_in_title"])
    l_contagious_in_title.append(VIDID_TO_FEATURES[vid_id]["contagious_in_title"])
    l_gobble_in_title.append(VIDID_TO_FEATURES[vid_id]["gobble_in_title"])
    l_dinde_in_title.append(VIDID_TO_FEATURES[vid_id]["dinde_in_title"])
    
    l_counts_bearded.append(VIDID_TO_FEATURES[vid_id]["counts_bearded"])
    l_counts_bow.append(VIDID_TO_FEATURES[vid_id]["counts_bow"])
    l_counts_gobbler.append(VIDID_TO_FEATURES[vid_id]["counts_gobbler"])
    l_counts_guajolote.append(VIDID_TO_FEATURES[vid_id]["counts_guajolote"])
    l_counts_pavo.append(VIDID_TO_FEATURES[vid_id]["counts_pavo"])
    l_counts_season.append(VIDID_TO_FEATURES[vid_id]["counts_season"])
    l_counts_chuckle.append(VIDID_TO_FEATURES[vid_id]["counts_chuckle"])
    l_counts_giggle.append(VIDID_TO_FEATURES[vid_id]["counts_giggle"])
    l_counts_tickle.append(VIDID_TO_FEATURES[vid_id]["counts_tickle"])
    l_counts_turkey.append(VIDID_TO_FEATURES[vid_id]["counts_turkey"])
    l_counts_wild.append(VIDID_TO_FEATURES[vid_id]["counts_wild"])
    l_counts_hunt.append(VIDID_TO_FEATURES[vid_id]["counts_hunt"])
    l_counts_laugh.append(VIDID_TO_FEATURES[vid_id]["counts_laugh"])
    
    l_counts_game.append(VIDID_TO_FEATURES[vid_id]["counts_game"])
    l_counts_rire.append(VIDID_TO_FEATURES[vid_id]["counts_rire"])
    l_counts_fou.append(VIDID_TO_FEATURES[vid_id]["counts_fou"])
    l_counts_funny.append(VIDID_TO_FEATURES[vid_id]["counts_funny"])
    l_counts_turkeys.append(VIDID_TO_FEATURES[vid_id]["counts_turkeys"])
    l_counts_animal.append(VIDID_TO_FEATURES[vid_id]["counts_animal"])
    l_counts_fun.append(VIDID_TO_FEATURES[vid_id]["counts_fun"])
    l_counts_laughter.append(VIDID_TO_FEATURES[vid_id]["counts_laughter"])
    l_counts_contagious.append(VIDID_TO_FEATURES[vid_id]["counts_contagious"])
    l_counts_gobble.append(VIDID_TO_FEATURES[vid_id]["counts_gobble"])
    l_counts_dinde.append(VIDID_TO_FEATURES[vid_id]["counts_dinde"])

    

TRAIN_DF["turkey_in_title"] = l_turkey_in_title
TRAIN_DF["hunt_in_title"] = l_hunt_in_title
TRAIN_DF["wild_in_title"] = l_wild_in_title
TRAIN_DF["laugh_in_title"] = l_laugh_in_title
TRAIN_DF["bearded_in_title"] = l_bearded_in_title
TRAIN_DF["bow_in_title"] = l_bow_in_title
TRAIN_DF["gobbler_in_title"] = l_gobbler_in_title
TRAIN_DF["guajolote_in_title"] = l_guajolote_in_title
TRAIN_DF["pavo_in_title"] = l_pavo_in_title
TRAIN_DF["season_in_title"] = l_season_in_title
TRAIN_DF["chuckle_in_title"] = l_chuckle_in_title
TRAIN_DF["giggle_in_title"] = l_giggle_in_title
TRAIN_DF["tickle_in_title"] = l_tickle_in_title

TRAIN_DF["game_in_title"] = l_game_in_title
TRAIN_DF["rire_in_title"] = l_rire_in_title
TRAIN_DF["fou_in_title"] = l_fou_in_title
TRAIN_DF["funny_in_title"] = l_funny_in_title
TRAIN_DF["turkeys_in_title"] = l_turkeys_in_title
TRAIN_DF["animal_in_title"] = l_animal_in_title
TRAIN_DF["fun_in_title"] = l_fun_in_title
TRAIN_DF["laughter_in_title"] = l_laughter_in_title
TRAIN_DF["contagious_in_title"] = l_contagious_in_title
TRAIN_DF["gobble_in_title"] = l_gobble_in_title
TRAIN_DF["dinde_in_title"] = l_dinde_in_title


TRAIN_DF["counts_turkey_zero"] = np.array(l_counts_turkey) == 0
TRAIN_DF["counts_wild_zero"] = np.array(l_counts_wild) == 0
TRAIN_DF["counts_hunt_zero"] = np.array(l_counts_hunt) == 0
TRAIN_DF["counts_laugh_zero"] = np.array(l_counts_laugh) == 0
TRAIN_DF["counts_bearded_zero"] = np.array(l_counts_bearded) == 0
TRAIN_DF["counts_bow_zero"] = np.array(l_counts_bow) == 0
TRAIN_DF["counts_gobbler_zero"] = np.array(l_counts_gobbler) == 0
TRAIN_DF["counts_guajolote_zero"] = np.array(l_counts_guajolote) == 0
TRAIN_DF["counts_pavo_zero"] = np.array(l_counts_pavo) == 0
TRAIN_DF["counts_season_zero"] = np.array(l_counts_season) == 0
TRAIN_DF["counts_chuckle_zero"] = np.array(l_counts_chuckle) == 0
TRAIN_DF["counts_giggle_zero"] = np.array(l_counts_giggle) == 0
TRAIN_DF["counts_tickle_zero"] = np.array(l_counts_tickle) == 0

TRAIN_DF["counts_game_zero"] = np.array(l_counts_game) == 0
TRAIN_DF["counts_rire_zero"] = np.array(l_counts_rire) == 0
TRAIN_DF["counts_fou_zero"] = np.array(l_counts_fou) == 0
TRAIN_DF["counts_funny_zero"] = np.array(l_counts_funny) == 0
TRAIN_DF["counts_turkeys_zero"] = np.array(l_counts_turkeys) == 0
TRAIN_DF["counts_animal_zero"] = np.array(l_counts_animal) == 0
TRAIN_DF["counts_fun_zero"] = np.array(l_counts_fun) == 0
TRAIN_DF["counts_laughter_zero"] = np.array(l_counts_laughter) == 0
TRAIN_DF["counts_contagious_zero"] = np.array(l_counts_contagious) == 0
TRAIN_DF["counts_gobble_zero"] = np.array(l_counts_gobble) == 0
TRAIN_DF["counts_dinde_zero"] = np.array(l_counts_dinde) == 0

TRAIN_DF["counts_turkey_less5"] = (np.array(l_counts_turkey) <= 5) & (np.array(l_counts_turkey) > 0)
TRAIN_DF["counts_turkey_more5"] = np.array(l_counts_turkey) > 5
TRAIN_DF["counts_wild_less5"] = (np.array(l_counts_wild) <= 5) & (np.array(l_counts_wild) > 0)
TRAIN_DF["counts_wild_more5"] = np.array(l_counts_wild) > 5
TRAIN_DF["counts_hunt_less5"] = (np.array(l_counts_hunt) <= 5) & (np.array(l_counts_hunt) > 0)
TRAIN_DF["counts_hunt_more5"] = np.array(l_counts_hunt) > 5
TRAIN_DF["counts_laugh_less5"] = (np.array(l_counts_laugh) <= 5) & (np.array(l_counts_laugh) > 0)
TRAIN_DF["counts_laugh_less5"] = np.array(l_counts_laugh) > 5

TRAIN_DF["counts_game_less5"] = (np.array(l_counts_game) <= 5) & (np.array(l_counts_game) > 0)
TRAIN_DF["counts_rire_less5"] = (np.array(l_counts_rire) <= 5) & (np.array(l_counts_rire) > 0)
TRAIN_DF["counts_fou_less5"] = (np.array(l_counts_fou) <= 5) & (np.array(l_counts_fou) > 0)
TRAIN_DF["counts_funny_less5"] = (np.array(l_counts_funny) <= 5) & (np.array(l_counts_funny) > 0)
TRAIN_DF["counts_turkeys_less5"] = (np.array(l_counts_turkeys) <= 5) & (np.array(l_counts_turkeys) > 0)
TRAIN_DF["counts_animal_less5"] = (np.array(l_counts_animal) <= 5) & (np.array(l_counts_animal) > 0)
TRAIN_DF["counts_fun_less5"] = (np.array(l_counts_fun) <= 5) & (np.array(l_counts_fun) > 0)
TRAIN_DF["counts_laughter_less5"] = (np.array(l_counts_laughter) <= 5) & (np.array(l_counts_laughter) > 0)
TRAIN_DF["counts_contagious_less5"] = (np.array(l_counts_contagious) <= 5) & (np.array(l_counts_contagious) > 0)
TRAIN_DF["counts_gobble_less5"] = (np.array(l_counts_gobble) <= 5) & (np.array(l_counts_gobble) > 0)
TRAIN_DF["counts_dinde_less5"] = (np.array(l_counts_dinde) <= 5) & (np.array(l_counts_dinde) > 0)

TRAIN_DF["counts_game_more5"] = np.array(l_counts_game) > 5
TRAIN_DF["counts_rire_more5"] = np.array(l_counts_rire) > 5
TRAIN_DF["counts_fou_more5"] = np.array(l_counts_fou) > 5
TRAIN_DF["counts_funny_more5"] = np.array(l_counts_funny) > 5
TRAIN_DF["counts_turkeys_more5"] = np.array(l_counts_turkeys) > 5
TRAIN_DF["counts_animal_more5"] = np.array(l_counts_animal) > 5
TRAIN_DF["counts_fun_more5"] = np.array(l_counts_fun) > 5
TRAIN_DF["counts_laughter_more5"] = np.array(l_counts_laughter)  > 5
TRAIN_DF["counts_contagious_more5"] = np.array(l_counts_contagious)  > 5
TRAIN_DF["counts_gobble_more5"] = np.array(l_counts_gobble) > 5
TRAIN_DF["counts_dinde_more5"] = np.array(l_counts_dinde)  > 5


TRAIN_DF["counts_bearded_less5"] = (np.array(l_counts_bearded) <= 5) & (np.array(l_counts_bearded) > 0)
TRAIN_DF["counts_bow_less5"] = (np.array(l_counts_bow) <= 5) & (np.array(l_counts_bow) > 0)
TRAIN_DF["counts_gobbler_less5"] = (np.array(l_counts_gobbler) <= 5) & (np.array(l_counts_gobbler) > 0)
TRAIN_DF["counts_guajolote_less5"] = (np.array(l_counts_guajolote) <= 5) & (np.array(l_counts_guajolote) > 0)
TRAIN_DF["counts_pavo_less5"] = (np.array(l_counts_pavo) <= 5) & (np.array(l_counts_pavo) > 0)
TRAIN_DF["counts_season_less5"] = (np.array(l_counts_season) <= 5) & (np.array(l_counts_season) > 0)
TRAIN_DF["counts_chuckle_less5"] = (np.array(l_counts_chuckle) <= 5) & (np.array(l_counts_chuckle) > 0)
TRAIN_DF["counts_giggle_less5"] = (np.array(l_counts_giggle) <= 5) & (np.array(l_counts_giggle) > 0)
TRAIN_DF["counts_tickle_less5"] = (np.array(l_counts_tickle) <= 5) & (np.array(l_counts_tickle) > 0)

TRAIN_DF["counts_bearded_more5"] = np.array(l_counts_bearded) > 5
TRAIN_DF["counts_bow_more5"] = np.array(l_counts_bow) > 5
TRAIN_DF["counts_gobbler_more5"] = np.array(l_counts_gobbler) > 5
TRAIN_DF["counts_guajolote_more5"] = np.array(l_counts_guajolote) > 5
TRAIN_DF["counts_pavo_more5"] = np.array(l_counts_pavo) > 5
TRAIN_DF["counts_season_more5"] = np.array(l_counts_season) > 5
TRAIN_DF["counts_chuckle_more5"] = np.array(l_counts_chuckle) > 5
TRAIN_DF["counts_giggle_more5"] = np.array(l_counts_giggle) > 5
TRAIN_DF["counts_tickle_more5"] = np.array(l_counts_tickle) > 5

TRAIN_DF["counts_turkey_more10"] = np.array(l_counts_turkey) > 10
TRAIN_DF["counts_wild_more10"] = np.array(l_counts_wild) > 10
TRAIN_DF["counts_hunt_more10"] = np.array(l_counts_hunt) > 10
TRAIN_DF["counts_laugh_more10"] = np.array(l_counts_laugh) > 10
TRAIN_DF["counts_bearded_more10"] = np.array(l_counts_bearded) > 10
TRAIN_DF["counts_bow_more10"] = np.array(l_counts_bow) > 10
TRAIN_DF["counts_gobbler_more10"] = np.array(l_counts_gobbler) > 10
TRAIN_DF["counts_guajolote_more10"] = np.array(l_counts_guajolote) > 10
TRAIN_DF["counts_pavo_more10"] = np.array(l_counts_pavo) > 10
TRAIN_DF["counts_season_more10"] = np.array(l_counts_season) > 10
TRAIN_DF["counts_chuckle_more10"] = np.array(l_counts_chuckle) > 10
TRAIN_DF["counts_giggle_more10"] = np.array(l_counts_giggle) > 10
TRAIN_DF["counts_tickle_more10"] = np.array(l_counts_tickle) > 10

TRAIN_DF["counts_game_more10"] = np.array(l_counts_game) > 10
TRAIN_DF["counts_rire_more10"] = np.array(l_counts_rire) > 10
TRAIN_DF["counts_fou_more10"] = np.array(l_counts_fou) > 10
TRAIN_DF["counts_funny_more10"] = np.array(l_counts_funny) > 10
TRAIN_DF["counts_turkeys_more10"] = np.array(l_counts_turkeys) > 10
TRAIN_DF["counts_animal_more10"] = np.array(l_counts_animal) > 10
TRAIN_DF["counts_fun_more10"] = np.array(l_counts_fun) > 10
TRAIN_DF["counts_laughter_more10"] = np.array(l_counts_laughter)  > 10
TRAIN_DF["counts_contagious_more10"] = np.array(l_counts_contagious)  > 10
TRAIN_DF["counts_gobble_more10"] = np.array(l_counts_gobble) > 10
TRAIN_DF["counts_dinde_more10"] = np.array(l_counts_dinde)  > 10


TRAIN_DF["counts_turkey_more15"] = np.array(l_counts_turkey) > 15
TRAIN_DF["counts_wild_more15"] = np.array(l_counts_wild) > 15
TRAIN_DF["counts_hunt_more15"] = np.array(l_counts_hunt) > 15
TRAIN_DF["counts_laugh_more15"] = np.array(l_counts_laugh) > 15
TRAIN_DF["counts_bearded_more15"] = np.array(l_counts_bearded) > 15
TRAIN_DF["counts_bow_more15"] = np.array(l_counts_bow) > 15
TRAIN_DF["counts_gobbler_more15"] = np.array(l_counts_gobbler) > 15
TRAIN_DF["counts_guajolote_more15"] = np.array(l_counts_guajolote) > 15
TRAIN_DF["counts_pavo_more15"] = np.array(l_counts_pavo) > 15
TRAIN_DF["counts_season_more15"] = np.array(l_counts_season) > 15
TRAIN_DF["counts_chuckle_more15"] = np.array(l_counts_chuckle) > 15
TRAIN_DF["counts_giggle_more15"] = np.array(l_counts_giggle) > 15
TRAIN_DF["counts_tickle_more15"] = np.array(l_counts_tickle) > 15


TRAIN_DF["counts_game_more15"] = np.array(l_counts_game) > 15
TRAIN_DF["counts_rire_more15"] = np.array(l_counts_rire) > 15
TRAIN_DF["counts_fou_more15"] = np.array(l_counts_fou) > 15
TRAIN_DF["counts_funny_more15"] = np.array(l_counts_funny) > 15
TRAIN_DF["counts_turkeys_more15"] = np.array(l_counts_turkeys) > 15
TRAIN_DF["counts_animal_more15"] = np.array(l_counts_animal) > 15
TRAIN_DF["counts_fun_more15"] = np.array(l_counts_fun) > 15
TRAIN_DF["counts_laughter_more15"] = np.array(l_counts_laughter)  > 15
TRAIN_DF["counts_contagious_more15"] = np.array(l_counts_contagious)  > 15
TRAIN_DF["counts_gobble_more15"] = np.array(l_counts_gobble) > 15
TRAIN_DF["counts_dinde_more15"] = np.array(l_counts_dinde)  > 15

TRAIN_DF["counts_game_more20"] = np.array(l_counts_game) > 20
TRAIN_DF["counts_rire_more20"] = np.array(l_counts_rire) > 20
TRAIN_DF["counts_fou_more20"] = np.array(l_counts_fou) > 20
TRAIN_DF["counts_funny_more20"] = np.array(l_counts_funny) > 20
TRAIN_DF["counts_turkeys_more20"] = np.array(l_counts_turkeys) > 20
TRAIN_DF["counts_animal_more20"] = np.array(l_counts_animal) > 20
TRAIN_DF["counts_fun_more20"] = np.array(l_counts_fun) > 20
TRAIN_DF["counts_laughter_more20"] = np.array(l_counts_laughter)  > 20
TRAIN_DF["counts_contagious_more20"] = np.array(l_counts_contagious)  > 20
TRAIN_DF["counts_gobble_more20"] = np.array(l_counts_gobble) > 20
TRAIN_DF["counts_dinde_more20"] = np.array(l_counts_dinde)  > 20

TRAIN_DF["counts_turkey_more20"] = np.array(l_counts_turkey) > 20
TRAIN_DF["counts_wild_more20"] = np.array(l_counts_wild) > 20
TRAIN_DF["counts_hunt_more20"] = np.array(l_counts_hunt) > 20
TRAIN_DF["counts_laugh_more20"] = np.array(l_counts_laugh) > 20
TRAIN_DF["counts_bearded_more20"] = np.array(l_counts_bearded) > 20
TRAIN_DF["counts_bow_more20"] = np.array(l_counts_bow) > 20
TRAIN_DF["counts_gobbler_more20"] = np.array(l_counts_gobbler) > 20
TRAIN_DF["counts_guajolote_more20"] = np.array(l_counts_guajolote) > 20
TRAIN_DF["counts_pavo_more20"] = np.array(l_counts_pavo) > 20
TRAIN_DF["counts_season_more20"] = np.array(l_counts_season) > 20
TRAIN_DF["counts_chuckle_more20"] = np.array(l_counts_chuckle) > 20
TRAIN_DF["counts_giggle_more20"] = np.array(l_counts_giggle) > 20
TRAIN_DF["counts_tickle_more20"] = np.array(l_counts_tickle) > 20



  0%|          | 0/1195 [00:00<?, ?it/s]

100%|██████████| 1195/1195 [00:00<00:00, 79304.35it/s]

In [334]:
TRAIN_DF[TRAIN_DF.vid_id == "1QrqkC1mt94"]['rire_in_title']

37    True
Name: rire_in_title, dtype: bool

In [335]:
l_turkey_in_title = []
l_hunt_in_title = []
l_wild_in_title = []
l_laugh_in_title = []
l_bearded_in_title = []
l_bow_in_title = []
l_gobbler_in_title = []
l_guajolote_in_title = []
l_pavo_in_title = []
l_season_in_title = []
l_chuckle_in_title = []
l_giggle_in_title = []
l_tickle_in_title = []
l_counts_turkey = []
l_counts_wild = []
l_counts_hunt = []
l_counts_laugh = []
l_counts_bearded = []
l_counts_bow = []
l_counts_gobbler = []
l_counts_guajolote = []
l_counts_pavo = []
l_counts_season = []
l_counts_chuckle = []
l_counts_giggle = []
l_counts_tickle = []

l_counts_game = []
l_counts_rire = []
l_counts_fou = []
l_counts_funny = []
l_counts_turkeys = []
l_counts_animal = []
l_counts_fun = []
l_counts_laughter = []
l_counts_contagious = []
l_counts_gobble = []
l_counts_dinde = []

l_game_in_title = []
l_rire_in_title = []
l_fou_in_title = []
l_funny_in_title = [] 
l_turkeys_in_title = []
l_animal_in_title = []
l_fun_in_title = []
l_laughter_in_title = []
l_contagious_in_title = []
l_gobble_in_title = []
l_dinde_in_title = []

for vid_id in tqdm(TEST_DF.vid_id.values):
 
    l_turkey_in_title.append(VIDID_TO_FEATURES[vid_id]["turkey_in_title"])
    l_hunt_in_title.append(VIDID_TO_FEATURES[vid_id]["hunt_in_title"])
    l_wild_in_title.append(VIDID_TO_FEATURES[vid_id]["wild_in_title"])
    l_laugh_in_title.append(VIDID_TO_FEATURES[vid_id]["laugh_in_title"])
    l_bearded_in_title.append(VIDID_TO_FEATURES[vid_id]["bearded_in_title"])
    l_bow_in_title.append(VIDID_TO_FEATURES[vid_id]["bow_in_title"])
    l_gobbler_in_title.append(VIDID_TO_FEATURES[vid_id]["gobbler_in_title"])
    l_guajolote_in_title.append(VIDID_TO_FEATURES[vid_id]["guajolote_in_title"])
    l_pavo_in_title.append(VIDID_TO_FEATURES[vid_id]["pavo_in_title"])
    l_season_in_title.append(VIDID_TO_FEATURES[vid_id]["season_in_title"])
    l_chuckle_in_title.append(VIDID_TO_FEATURES[vid_id]["chuckle_in_title"])
    l_giggle_in_title.append(VIDID_TO_FEATURES[vid_id]["giggle_in_title"])
    l_tickle_in_title.append(VIDID_TO_FEATURES[vid_id]["tickle_in_title"])
    
    l_game_in_title.append(VIDID_TO_FEATURES[vid_id]["game_in_title"])
    l_rire_in_title.append(VIDID_TO_FEATURES[vid_id]["rire_in_title"])
    l_fou_in_title.append(VIDID_TO_FEATURES[vid_id]["fou_in_title"])
    l_funny_in_title.append(VIDID_TO_FEATURES[vid_id]["funny_in_title"])
    l_turkeys_in_title.append(VIDID_TO_FEATURES[vid_id]["turkeys_in_title"])
    l_animal_in_title.append(VIDID_TO_FEATURES[vid_id]["animal_in_title"])
    l_fun_in_title.append(VIDID_TO_FEATURES[vid_id]["fun_in_title"])
    l_laughter_in_title.append(VIDID_TO_FEATURES[vid_id]["laughter_in_title"])
    l_contagious_in_title.append(VIDID_TO_FEATURES[vid_id]["contagious_in_title"])
    l_gobble_in_title.append(VIDID_TO_FEATURES[vid_id]["gobble_in_title"])
    l_dinde_in_title.append(VIDID_TO_FEATURES[vid_id]["dinde_in_title"])
    
    
    l_counts_bearded.append(VIDID_TO_FEATURES[vid_id]["counts_bearded"])
    l_counts_bow.append(VIDID_TO_FEATURES[vid_id]["counts_bow"])
    l_counts_gobbler.append(VIDID_TO_FEATURES[vid_id]["counts_gobbler"])
    l_counts_guajolote.append(VIDID_TO_FEATURES[vid_id]["counts_guajolote"])
    l_counts_pavo.append(VIDID_TO_FEATURES[vid_id]["counts_pavo"])
    l_counts_season.append(VIDID_TO_FEATURES[vid_id]["counts_season"])
    l_counts_chuckle.append(VIDID_TO_FEATURES[vid_id]["counts_chuckle"])
    l_counts_giggle.append(VIDID_TO_FEATURES[vid_id]["counts_giggle"])
    l_counts_tickle.append(VIDID_TO_FEATURES[vid_id]["counts_tickle"])
    l_counts_turkey.append(VIDID_TO_FEATURES[vid_id]["counts_turkey"])
    l_counts_wild.append(VIDID_TO_FEATURES[vid_id]["counts_wild"])
    l_counts_hunt.append(VIDID_TO_FEATURES[vid_id]["counts_hunt"])
    l_counts_laugh.append(VIDID_TO_FEATURES[vid_id]["counts_laugh"])
    
    l_counts_game.append(VIDID_TO_FEATURES[vid_id]["counts_game"])
    l_counts_rire.append(VIDID_TO_FEATURES[vid_id]["counts_rire"])
    l_counts_fou.append(VIDID_TO_FEATURES[vid_id]["counts_fou"])
    l_counts_funny.append(VIDID_TO_FEATURES[vid_id]["counts_funny"])
    l_counts_turkeys.append(VIDID_TO_FEATURES[vid_id]["counts_turkeys"])
    l_counts_animal.append(VIDID_TO_FEATURES[vid_id]["counts_animal"])
    l_counts_fun.append(VIDID_TO_FEATURES[vid_id]["counts_fun"])
    l_counts_laughter.append(VIDID_TO_FEATURES[vid_id]["counts_laughter"])
    l_counts_contagious.append(VIDID_TO_FEATURES[vid_id]["counts_contagious"])
    l_counts_gobble.append(VIDID_TO_FEATURES[vid_id]["counts_gobble"])
    l_counts_dinde.append(VIDID_TO_FEATURES[vid_id]["counts_dinde"])

    

TEST_DF["turkey_in_title"] = l_turkey_in_title
TEST_DF["hunt_in_title"] = l_hunt_in_title
TEST_DF["wild_in_title"] = l_wild_in_title
TEST_DF["laugh_in_title"] = l_laugh_in_title
TEST_DF["bearded_in_title"] = l_bearded_in_title
TEST_DF["bow_in_title"] = l_bow_in_title
TEST_DF["gobbler_in_title"] = l_gobbler_in_title
TEST_DF["guajolote_in_title"] = l_guajolote_in_title
TEST_DF["pavo_in_title"] = l_pavo_in_title
TEST_DF["season_in_title"] = l_season_in_title
TEST_DF["chuckle_in_title"] = l_chuckle_in_title
TEST_DF["giggle_in_title"] = l_giggle_in_title
TEST_DF["tickle_in_title"] = l_tickle_in_title

TEST_DF["game_in_title"] = l_game_in_title
TEST_DF["rire_in_title"] = l_rire_in_title
TEST_DF["fou_in_title"] = l_fou_in_title
TEST_DF["funny_in_title"] = l_funny_in_title
TEST_DF["turkeys_in_title"] = l_turkeys_in_title
TEST_DF["animal_in_title"] = l_animal_in_title
TEST_DF["fun_in_title"] = l_fun_in_title
TEST_DF["laughter_in_title"] = l_laughter_in_title
TEST_DF["contagious_in_title"] = l_contagious_in_title
TEST_DF["gobble_in_title"] = l_gobble_in_title
TEST_DF["dinde_in_title"] = l_dinde_in_title


TEST_DF["counts_turkey_zero"] = np.array(l_counts_turkey) == 0
TEST_DF["counts_wild_zero"] = np.array(l_counts_wild) == 0
TEST_DF["counts_hunt_zero"] = np.array(l_counts_hunt) == 0
TEST_DF["counts_laugh_zero"] = np.array(l_counts_laugh) == 0
TEST_DF["counts_bearded_zero"] = np.array(l_counts_bearded) == 0
TEST_DF["counts_bow_zero"] = np.array(l_counts_bow) == 0
TEST_DF["counts_gobbler_zero"] = np.array(l_counts_gobbler) == 0
TEST_DF["counts_guajolote_zero"] = np.array(l_counts_guajolote) == 0
TEST_DF["counts_pavo_zero"] = np.array(l_counts_pavo) == 0
TEST_DF["counts_season_zero"] = np.array(l_counts_season) == 0
TEST_DF["counts_chuckle_zero"] = np.array(l_counts_chuckle) == 0
TEST_DF["counts_giggle_zero"] = np.array(l_counts_giggle) == 0
TEST_DF["counts_tickle_zero"] = np.array(l_counts_tickle) == 0

TEST_DF["counts_game_zero"] = np.array(l_counts_game) == 0
TEST_DF["counts_rire_zero"] = np.array(l_counts_rire) == 0
TEST_DF["counts_fou_zero"] = np.array(l_counts_fou) == 0
TEST_DF["counts_funny_zero"] = np.array(l_counts_funny) == 0
TEST_DF["counts_turkeys_zero"] = np.array(l_counts_turkeys) == 0
TEST_DF["counts_animal_zero"] = np.array(l_counts_animal) == 0
TEST_DF["counts_fun_zero"] = np.array(l_counts_fun) == 0
TEST_DF["counts_laughter_zero"] = np.array(l_counts_laughter) == 0
TEST_DF["counts_contagious_zero"] = np.array(l_counts_contagious) == 0
TEST_DF["counts_gobble_zero"] = np.array(l_counts_gobble) == 0
TEST_DF["counts_dinde_zero"] = np.array(l_counts_dinde) == 0

TEST_DF["counts_turkey_less5"] = (np.array(l_counts_turkey) <= 5) & (np.array(l_counts_turkey) > 0)
TEST_DF["counts_turkey_more5"] = np.array(l_counts_turkey) > 5
TEST_DF["counts_wild_less5"] = (np.array(l_counts_wild) <= 5) & (np.array(l_counts_wild) > 0)
TEST_DF["counts_wild_more5"] = np.array(l_counts_wild) > 5
TEST_DF["counts_hunt_less5"] = (np.array(l_counts_hunt) <= 5) & (np.array(l_counts_hunt) > 0)
TEST_DF["counts_hunt_more5"] = np.array(l_counts_hunt) > 5
TEST_DF["counts_laugh_less5"] = (np.array(l_counts_laugh) <= 5) & (np.array(l_counts_laugh) > 0)
TEST_DF["counts_laugh_more5"] = np.array(l_counts_laugh) > 5


TEST_DF["counts_game_less5"] = (np.array(l_counts_game) <= 5) & (np.array(l_counts_game) > 0)
TEST_DF["counts_rire_less5"] = (np.array(l_counts_rire) <= 5) & (np.array(l_counts_rire) > 0)
TEST_DF["counts_fou_less5"] = (np.array(l_counts_fou) <= 5) & (np.array(l_counts_fou) > 0)
TEST_DF["counts_funny_less5"] = (np.array(l_counts_funny) <= 5) & (np.array(l_counts_funny) > 0)
TEST_DF["counts_turkeys_less5"] = (np.array(l_counts_turkeys) <= 5) & (np.array(l_counts_turkeys) > 0)
TEST_DF["counts_animal_less5"] = (np.array(l_counts_animal) <= 5) & (np.array(l_counts_animal) > 0)
TEST_DF["counts_fun_less5"] = (np.array(l_counts_fun) <= 5) & (np.array(l_counts_fun) > 0)
TEST_DF["counts_laughter_less5"] = (np.array(l_counts_laughter) <= 5) & (np.array(l_counts_laughter) > 0)
TEST_DF["counts_contagious_less5"] = (np.array(l_counts_contagious) <= 5) & (np.array(l_counts_contagious) > 0)
TEST_DF["counts_gobble_less5"] = (np.array(l_counts_gobble) <= 5) & (np.array(l_counts_gobble) > 0)
TEST_DF["counts_dinde_less5"] = (np.array(l_counts_dinde) <= 5) & (np.array(l_counts_dinde) > 0)

TEST_DF["counts_game_more5"] = np.array(l_counts_game) > 5
TEST_DF["counts_rire_more5"] = np.array(l_counts_rire) > 5
TEST_DF["counts_fou_more5"] = np.array(l_counts_fou) > 5
TEST_DF["counts_funny_more5"] = np.array(l_counts_funny) > 5
TEST_DF["counts_turkeys_more5"] = np.array(l_counts_turkeys) > 5
TEST_DF["counts_animal_more5"] = np.array(l_counts_animal) > 5
TEST_DF["counts_fun_more5"] = np.array(l_counts_fun) > 5
TEST_DF["counts_laughter_more5"] = np.array(l_counts_laughter)  > 5
TEST_DF["counts_contagious_more5"] = np.array(l_counts_contagious)  > 5
TEST_DF["counts_gobble_more5"] = np.array(l_counts_gobble) > 5
TEST_DF["counts_dinde_more5"] = np.array(l_counts_dinde)  > 5


TEST_DF["counts_bearded_less5"] = (np.array(l_counts_bearded) <= 5) & (np.array(l_counts_bearded) > 0)
TEST_DF["counts_bow_less5"] = (np.array(l_counts_bow) <= 5) & (np.array(l_counts_bow) > 0)
TEST_DF["counts_gobbler_less5"] = (np.array(l_counts_gobbler) <= 5) & (np.array(l_counts_gobbler) > 0)
TEST_DF["counts_guajolote_less5"] = (np.array(l_counts_guajolote) <= 5) & (np.array(l_counts_guajolote) > 0)
TEST_DF["counts_pavo_less5"] = (np.array(l_counts_pavo) <= 5) & (np.array(l_counts_pavo) > 0)
TEST_DF["counts_season_less5"] = (np.array(l_counts_season) <= 5) & (np.array(l_counts_season) > 0)
TEST_DF["counts_chuckle_less5"] = (np.array(l_counts_chuckle) <= 5) & (np.array(l_counts_chuckle) > 0)
TEST_DF["counts_giggle_less5"] = (np.array(l_counts_giggle) <= 5) & (np.array(l_counts_giggle) > 0)
TEST_DF["counts_tickle_less5"] = (np.array(l_counts_tickle) <= 5) & (np.array(l_counts_tickle) > 0)

TEST_DF["counts_bearded_more5"] = np.array(l_counts_bearded) > 5
TEST_DF["counts_bow_more5"] = np.array(l_counts_bow) > 5
TEST_DF["counts_gobbler_more5"] = np.array(l_counts_gobbler) > 5
TEST_DF["counts_guajolote_more5"] = np.array(l_counts_guajolote) > 5
TEST_DF["counts_pavo_more5"] = np.array(l_counts_pavo) > 5
TEST_DF["counts_season_more5"] = np.array(l_counts_season) > 5
TEST_DF["counts_chuckle_more5"] = np.array(l_counts_chuckle) > 5
TEST_DF["counts_giggle_more5"] = np.array(l_counts_giggle) > 5
TEST_DF["counts_tickle_more5"] = np.array(l_counts_tickle) > 5

TEST_DF["counts_turkey_more10"] = np.array(l_counts_turkey) > 10
TEST_DF["counts_wild_more10"] = np.array(l_counts_wild) > 10
TEST_DF["counts_hunt_more10"] = np.array(l_counts_hunt) > 10
TEST_DF["counts_laugh_more10"] = np.array(l_counts_laugh) > 10
TEST_DF["counts_bearded_more10"] = np.array(l_counts_bearded) > 10
TEST_DF["counts_bow_more10"] = np.array(l_counts_bow) > 10
TEST_DF["counts_gobbler_more10"] = np.array(l_counts_gobbler) > 10
TEST_DF["counts_guajolote_more10"] = np.array(l_counts_guajolote) > 10
TEST_DF["counts_pavo_more10"] = np.array(l_counts_pavo) > 10
TEST_DF["counts_season_more10"] = np.array(l_counts_season) > 10
TEST_DF["counts_chuckle_more10"] = np.array(l_counts_chuckle) > 10
TEST_DF["counts_giggle_more10"] = np.array(l_counts_giggle) > 10
TEST_DF["counts_tickle_more10"] = np.array(l_counts_tickle) > 10


TEST_DF["counts_game_more10"] = np.array(l_counts_game) > 10
TEST_DF["counts_rire_more10"] = np.array(l_counts_rire) > 10
TEST_DF["counts_fou_more10"] = np.array(l_counts_fou) > 10
TEST_DF["counts_funny_more10"] = np.array(l_counts_funny) > 10
TEST_DF["counts_turkeys_more10"] = np.array(l_counts_turkeys) > 10
TEST_DF["counts_animal_more10"] = np.array(l_counts_animal) > 10
TEST_DF["counts_fun_more10"] = np.array(l_counts_fun) > 10
TEST_DF["counts_laughter_more10"] = np.array(l_counts_laughter)  > 10
TEST_DF["counts_contagious_more10"] = np.array(l_counts_contagious)  > 10
TEST_DF["counts_gobble_more10"] = np.array(l_counts_gobble) > 10
TEST_DF["counts_dinde_more10"] = np.array(l_counts_dinde)  > 10


TEST_DF["counts_turkey_more15"] = np.array(l_counts_turkey) > 15
TEST_DF["counts_wild_more15"] = np.array(l_counts_wild) > 15
TEST_DF["counts_hunt_more15"] = np.array(l_counts_hunt) > 15
TEST_DF["counts_laugh_more15"] = np.array(l_counts_laugh) > 15
TEST_DF["counts_bearded_more15"] = np.array(l_counts_bearded) > 15
TEST_DF["counts_bow_more15"] = np.array(l_counts_bow) > 15
TEST_DF["counts_gobbler_more15"] = np.array(l_counts_gobbler) > 15
TEST_DF["counts_guajolote_more15"] = np.array(l_counts_guajolote) > 15
TEST_DF["counts_pavo_more15"] = np.array(l_counts_pavo) > 15
TEST_DF["counts_season_more15"] = np.array(l_counts_season) > 15
TEST_DF["counts_chuckle_more15"] = np.array(l_counts_chuckle) > 15
TEST_DF["counts_giggle_more15"] = np.array(l_counts_giggle) > 15
TEST_DF["counts_tickle_more15"] = np.array(l_counts_tickle) > 15

TEST_DF["counts_game_more15"] = np.array(l_counts_game) > 15
TEST_DF["counts_rire_more15"] = np.array(l_counts_rire) > 15
TEST_DF["counts_fou_more15"] = np.array(l_counts_fou) > 15
TEST_DF["counts_funny_more15"] = np.array(l_counts_funny) > 15
TEST_DF["counts_turkeys_more15"] = np.array(l_counts_turkeys) > 15
TEST_DF["counts_animal_more15"] = np.array(l_counts_animal) > 15
TEST_DF["counts_fun_more15"] = np.array(l_counts_fun) > 15
TEST_DF["counts_laughter_more15"] = np.array(l_counts_laughter)  > 15
TEST_DF["counts_contagious_more15"] = np.array(l_counts_contagious)  > 15
TEST_DF["counts_gobble_more15"] = np.array(l_counts_gobble) > 15
TEST_DF["counts_dinde_more15"] = np.array(l_counts_dinde)  > 15

TEST_DF["counts_game_more20"] = np.array(l_counts_game) > 20
TEST_DF["counts_rire_more20"] = np.array(l_counts_rire) > 20
TEST_DF["counts_fou_more20"] = np.array(l_counts_fou) > 20
TEST_DF["counts_funny_more20"] = np.array(l_counts_funny) > 20
TEST_DF["counts_turkeys_more20"] = np.array(l_counts_turkeys) > 20
TEST_DF["counts_animal_more20"] = np.array(l_counts_animal) > 20
TEST_DF["counts_fun_more20"] = np.array(l_counts_fun) > 20
TEST_DF["counts_laughter_more20"] = np.array(l_counts_laughter)  > 20
TEST_DF["counts_contagious_more20"] = np.array(l_counts_contagious)  > 20
TEST_DF["counts_gobble_more20"] = np.array(l_counts_gobble) > 20
TEST_DF["counts_dinde_more20"] = np.array(l_counts_dinde)  > 20


TEST_DF["counts_turkey_more20"] = np.array(l_counts_turkey) > 20
TEST_DF["counts_wild_more20"] = np.array(l_counts_wild) > 20
TEST_DF["counts_hunt_more20"] = np.array(l_counts_hunt) > 20
TEST_DF["counts_laugh_more20"] = np.array(l_counts_laugh) > 20
TEST_DF["counts_bearded_more20"] = np.array(l_counts_bearded) > 20
TEST_DF["counts_bow_more20"] = np.array(l_counts_bow) > 20
TEST_DF["counts_gobbler_more20"] = np.array(l_counts_gobbler) > 20
TEST_DF["counts_guajolote_more20"] = np.array(l_counts_guajolote) > 20
TEST_DF["counts_pavo_more20"] = np.array(l_counts_pavo) > 20
TEST_DF["counts_season_more20"] = np.array(l_counts_season) > 20
TEST_DF["counts_chuckle_more20"] = np.array(l_counts_chuckle) > 20
TEST_DF["counts_giggle_more20"] = np.array(l_counts_giggle) > 20
TEST_DF["counts_tickle_more20"] = np.array(l_counts_tickle) > 20



  0%|          | 0/1196 [00:00<?, ?it/s]

100%|██████████| 1196/1196 [00:00<00:00, 79109.11it/s]

## Making Inferences

In [336]:
features = [
    
    "game_in_title",
    "rire_in_title",
    "fou_in_title",
    "funny_in_title",
    "turkeys_in_title",
    "animal_in_title",
    "fun_in_title",
    "laughter_in_title",
    "contagious_in_title",
    "gobble_in_title",
    "dinde_in_title",
    'counts_game_zero',
    'counts_game_zero',
    'counts_game_less5',
    'counts_game_more5',
    'counts_game_more10',
    'counts_game_more15',
    'counts_game_more20',
    'counts_rire_zero',
    'counts_rire_less5',
    'counts_rire_more5',
    'counts_rire_more10',
    'counts_rire_more15',
    'counts_rire_more20',
    'counts_fou_zero',
    'counts_fou_less5',
    'counts_fou_more5',
    'counts_fou_more10',
    'counts_fou_more15',
    'counts_fou_more20',
    'counts_funny_zero',
    'counts_funny_less5',
    'counts_funny_more5',
    'counts_funny_more10',
    'counts_funny_more15',
    'counts_funny_more20',
    'counts_turkeys_zero',
    'counts_turkeys_less5',
    'counts_turkeys_more5',
    'counts_turkeys_more10',
    'counts_turkeys_more15',
    'counts_turkeys_more20',
    'counts_animal_zero',
    'counts_animal_less5',
    'counts_animal_more5',
    'counts_animal_more10',
    'counts_animal_more15',
    'counts_animal_more20',
    'counts_fun_zero',
    'counts_fun_less5',
    'counts_fun_more5',
    'counts_fun_more10',
    'counts_fun_more15',
    'counts_fun_more20',
    'counts_laughter_zero',
    'counts_laughter_less5',
    'counts_laughter_more5',
    'counts_laughter_more10',
    'counts_laughter_more15',
    'counts_laughter_more20',
    'counts_contagious_zero',
    'counts_contagious_less5',
    'counts_contagious_more5',
    'counts_contagious_more10',
    'counts_contagious_more15',
    'counts_contagious_more20',
    'counts_gobble_zero',
    'counts_gobble_less5',
    'counts_gobble_more5',
    'counts_gobble_more10',
    'counts_gobble_more15',
    'counts_gobble_more20',
    'counts_dinde_zero',
    'counts_dinde_less5',
    'counts_dinde_more5',
    'counts_dinde_more10',
    'counts_dinde_more15',
    'counts_dinde_more20',
    "turkey_in_title", 
    "hunt_in_title", 
    "wild_in_title", 
    "laugh_in_title", 
    "bearded_in_title",
    "bow_in_title",
    "gobbler_in_title",
    "guajolote_in_title",
    "pavo_in_title",
    "chuckle_in_title",
    "giggle_in_title",
    "tickle_in_title",
    "counts_turkey_zero",
    "counts_wild_zero",
    "counts_hunt_zero",
    "counts_laugh_zero",
    "counts_bearded_zero",
    "counts_bow_zero",
    "counts_gobbler_zero",
    "counts_guajolote_zero",
    "counts_pavo_zero",
    "counts_chuckle_zero",
    "counts_giggle_zero",
    "counts_tickle_zero",
    "counts_turkey_less5", 
    "counts_wild_less5", 
    "counts_hunt_less5", 
    "counts_laugh_less5",
    "counts_turkey_more5", 
    "counts_wild_more5", 
    "counts_hunt_more5", 
    "counts_laugh_more5",
    "counts_bearded_less5",
    "counts_bow_less5",
    "counts_gobbler_less5",
    "counts_guajolote_less5",
    "counts_pavo_less5",
    "counts_chuckle_less5",
    "counts_giggle_less5",
    "counts_tickle_less5",
    "counts_bearded_more5",
    "counts_bow_more5",
    "counts_gobbler_more5",
    "counts_guajolote_more5",
    "counts_pavo_more5",
    "counts_chuckle_more5",
    "counts_giggle_more5",
    "counts_tickle_more5",
    "counts_turkey_more10", 
    "counts_wild_more10", 
    "counts_hunt_more10", 
    "counts_laugh_more10",
    "counts_bearded_more10",
    "counts_bow_more10",
    "counts_gobbler_more10",
    "counts_guajolote_more10",
    "counts_pavo_more10",
    "counts_chuckle_more10",
    "counts_giggle_more10",
    "counts_tickle_more10",
    "counts_turkey_more15", 
    "counts_wild_more15", 
    "counts_hunt_more15", 
    "counts_laugh_more15",
    "counts_bearded_more15",
    "counts_bow_more15",
    "counts_gobbler_more15",
    "counts_guajolote_more15",
    "counts_pavo_more15",
    "counts_chuckle_more15",
    "counts_giggle_more15",
    "counts_tickle_more15",
    "counts_turkey_more20",
    "counts_wild_more20",
    "counts_hunt_more20",
    "counts_laugh_more20",
    "counts_bearded_more20",
    "counts_bow_more20",
    "counts_gobbler_more20",
    "counts_guajolote_more20",
    "counts_pavo_more20",
    "counts_chuckle_more20",
    "counts_giggle_more20",
    "counts_tickle_more20",
]

In [337]:
TRAIN_DF["is_not_turkey"] = TRAIN_DF["is_turkey"].values == 0

In [340]:
for feature in features:
    if len(TRAIN_DF[TRAIN_DF[feature]==True]) != 0 and len(TRAIN_DF[TRAIN_DF[feature]==False]) != 0:
        if sum(TRAIN_DF[TRAIN_DF[feature]==True]["is_turkey"].values)/len(TRAIN_DF[TRAIN_DF[feature]==True])==1.0:
            print("If {} == True => 1 | {} items".format(feature, len(TRAIN_DF[TRAIN_DF[feature]==True])))
            
        if sum(TRAIN_DF[TRAIN_DF[feature]==False]["is_turkey"].values)/len(TRAIN_DF[TRAIN_DF[feature]==False])==1.0:
            print("If {} == False => 1 | {} items".format(feature, len(TRAIN_DF[TRAIN_DF[feature]==False])))
        if sum(TRAIN_DF[TRAIN_DF[feature]==True]["is_not_turkey"].values)/len(TRAIN_DF[TRAIN_DF[feature]==True])==1.0:
            print("If {} == True => 0 | {} items".format(feature, len(TRAIN_DF[TRAIN_DF[feature]==True])))
        if sum(TRAIN_DF[TRAIN_DF[feature]==False]["is_not_turkey"].values)/len(TRAIN_DF[TRAIN_DF[feature]==False])==1.0:
            print("If {} == False => 0 | {} items".format(feature, len(TRAIN_DF[TRAIN_DF[feature]==False])))

If rire_in_title == True => 0 | 2 items
If turkeys_in_title == True => 1 | 21 items
If contagious_in_title == True => 0 | 7 items
If gobble_in_title == True => 1 | 39 items
If dinde_in_title == True => 1 | 2 items
If counts_game_more10 == True => 0 | 1 items
If counts_rire_more5 == True => 0 | 2 items
If counts_rire_more10 == True => 0 | 2 items
If counts_rire_more15 == True => 0 | 1 items
If counts_fou_zero == False => 0 | 5 items
If counts_fou_less5 == True => 0 | 4 items
If counts_fou_more5 == True => 0 | 1 items
If counts_funny_more15 == True => 0 | 3 items
If counts_funny_more20 == True => 0 | 2 items
If counts_turkeys_more5 == True => 1 | 28 items
If counts_turkeys_more10 == True => 1 | 3 items
If counts_fun_more5 == True => 0 | 1 items
If counts_laughter_zero == False => 0 | 279 items
If counts_laughter_less5 == True => 0 | 181 items
If counts_laughter_more5 == True => 0 | 98 items
If counts_laughter_more10 == True => 0 | 43 items
If counts_laughter_more15 == True => 0 | 23 item

In [341]:
import random
def generate_double_rainbow():
    print("def unicornify(x):\n")
    print("    if True == False: pass") ## just to avoid checking for the first if statement so that we can loop over elifs 
    for feature in features:
        if len(TRAIN_DF[TRAIN_DF[feature]==True]) != 0 and len(TRAIN_DF[TRAIN_DF[feature]==False]) != 0:
            if sum(TRAIN_DF[TRAIN_DF[feature]==True]["is_turkey"].values)/len(TRAIN_DF[TRAIN_DF[feature]==True])==1.0:
                print("    elif x['{}'] == True: return 1".format(feature))
            if sum(TRAIN_DF[TRAIN_DF[feature]==False]["is_turkey"].values)/len(TRAIN_DF[TRAIN_DF[feature]==False])==1.0:
                print("    elif x['{}'] == False: return 1".format(feature))

            if sum(TRAIN_DF[TRAIN_DF[feature]==True]["is_not_turkey"].values)/len(TRAIN_DF[TRAIN_DF[feature]==True])==1.0:
                print("    elif x['{}'] == True: return 0".format(feature))
            if sum(TRAIN_DF[TRAIN_DF[feature]==False]["is_not_turkey"].values)/len(TRAIN_DF[TRAIN_DF[feature]==False])==1.0:
                print("    elif x['{}'] == False: return 0".format(feature))
    print("    return None")
    print("".join([ random.choice(["🦄", "🌈"]) for _ in range(42)]))


In [342]:
generate_double_rainbow()

def unicornify(x):

    if True == False: pass
    elif x['rire_in_title'] == True: return 0
    elif x['turkeys_in_title'] == True: return 1
    elif x['contagious_in_title'] == True: return 0
    elif x['gobble_in_title'] == True: return 1
    elif x['dinde_in_title'] == True: return 1
    elif x['counts_game_more10'] == True: return 0
    elif x['counts_rire_more5'] == True: return 0
    elif x['counts_rire_more10'] == True: return 0
    elif x['counts_rire_more15'] == True: return 0
    elif x['counts_fou_zero'] == False: return 0
    elif x['counts_fou_less5'] == True: return 0
    elif x['counts_fou_more5'] == True: return 0
    elif x['counts_funny_more15'] == True: return 0
    elif x['counts_funny_more20'] == True: return 0
    elif x['counts_turkeys_more5'] == True: return 1
    elif x['counts_turkeys_more10'] == True: return 1
    elif x['counts_fun_more5'] == True: return 0
    elif x['counts_laughter_zero'] == False: return 0
    elif x['counts_laughter_less5'] == True: re

In [344]:
 def unicornify(x):
    if True == False: pass
    elif x['rire_in_title'] == True: return 0
    elif x['turkeys_in_title'] == True: return 1
    elif x['contagious_in_title'] == True: return 0
    elif x['gobble_in_title'] == True: return 1
    elif x['dinde_in_title'] == True: return 1
    elif x['counts_game_more10'] == True: return 0
    elif x['counts_rire_more5'] == True: return 0
    elif x['counts_rire_more10'] == True: return 0
    elif x['counts_rire_more15'] == True: return 0
    elif x['counts_fou_zero'] == False: return 0
    elif x['counts_fou_less5'] == True: return 0
    elif x['counts_fou_more5'] == True: return 0
    elif x['counts_funny_more15'] == True: return 0
    elif x['counts_funny_more20'] == True: return 0
    elif x['counts_turkeys_more5'] == True: return 1
    elif x['counts_turkeys_more10'] == True: return 1
    elif x['counts_fun_more5'] == True: return 0
    elif x['counts_laughter_zero'] == False: return 0
    elif x['counts_laughter_less5'] == True: return 0
    elif x['counts_laughter_more5'] == True: return 0
    elif x['counts_laughter_more10'] == True: return 0
    elif x['counts_laughter_more15'] == True: return 0
    elif x['counts_laughter_more20'] == True: return 0
    elif x['counts_contagious_zero'] == False: return 0
    elif x['counts_contagious_less5'] == True: return 0
    elif x['counts_contagious_more5'] == True: return 0
    elif x['counts_contagious_more10'] == True: return 0
    elif x['counts_gobble_zero'] == False: return 1
    elif x['counts_gobble_less5'] == True: return 1
    elif x['counts_gobble_more5'] == True: return 1
    elif x['counts_gobble_more10'] == True: return 1
    elif x['counts_dinde_zero'] == False: return 1
    elif x['counts_dinde_less5'] == True: return 1
    elif x['turkey_in_title'] == True: return 1
    elif x['bearded_in_title'] == True: return 1
    elif x['gobbler_in_title'] == True: return 1
    elif x['guajolote_in_title'] == True: return 1
    elif x['pavo_in_title'] == True: return 1
    elif x['chuckle_in_title'] == True: return 0
    elif x['giggle_in_title'] == True: return 0
    elif x['counts_gobbler_zero'] == False: return 1
    elif x['counts_guajolote_zero'] == False: return 1
    elif x['counts_pavo_zero'] == False: return 1
    elif x['counts_chuckle_zero'] == False: return 0
    elif x['counts_giggle_zero'] == False: return 0
    elif x['counts_laugh_less5'] == True: return 0
    elif x['counts_turkey_more5'] == True: return 1
    elif x['counts_hunt_more5'] == True: return 1
    elif x['counts_laugh_more5'] == True: return 0
    elif x['counts_gobbler_less5'] == True: return 1
    elif x['counts_guajolote_less5'] == True: return 1
    elif x['counts_pavo_less5'] == True: return 1
    elif x['counts_chuckle_less5'] == True: return 0
    elif x['counts_giggle_less5'] == True: return 0
    elif x['counts_tickle_less5'] == True: return 0
    elif x['counts_bearded_more5'] == True: return 1
    elif x['counts_bow_more5'] == True: return 1
    elif x['counts_gobbler_more5'] == True: return 1
    elif x['counts_guajolote_more5'] == True: return 1
    elif x['counts_pavo_more5'] == True: return 1
    elif x['counts_chuckle_more5'] == True: return 0
    elif x['counts_tickle_more5'] == True: return 1
    elif x['counts_turkey_more10'] == True: return 1
    elif x['counts_wild_more10'] == True: return 0
    elif x['counts_hunt_more10'] == True: return 1
    elif x['counts_bow_more10'] == True: return 1
    elif x['counts_pavo_more10'] == True: return 1
    elif x['counts_chuckle_more10'] == True: return 0
    elif x['counts_tickle_more10'] == True: return 1
    elif x['counts_turkey_more15'] == True: return 1
    elif x['counts_wild_more15'] == True: return 0
    elif x['counts_bow_more15'] == True: return 1
    elif x['counts_chuckle_more15'] == True: return 0
    elif x['counts_tickle_more15'] == True: return 1
    elif x['counts_turkey_more20'] == True: return 1
    elif x['counts_wild_more20'] == True: return 0
    elif x['counts_chuckle_more20'] == True: return 0
    return None

In [372]:
def train_model(X_train, y_train, X_test, n_splits=5, verbose=2):

    missfits = [] ## missclassified items
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=424242)
    preds = []
    historys = []
    fold = 0
    aucs = 0.0
    threshold = 0.5

    for train_idx, val_idx in kf.split(X_train):
        x_train_f = X_train[train_idx]
        y_train_f = y_train[train_idx]
        x_val_f = X_train[val_idx]
        y_val_f = y_train[val_idx]

        model = get_model(neurons=128, momentum=0.6, learn_rate=0.001, dropout_rate2=0.1, dropout_rate1=0.1)
        history = model.fit(x_train_f, y_train_f,
            batch_size=128,
            epochs=20,
            verbose = verbose,
            validation_data=(x_val_f, y_val_f),
            callbacks = [])
        
        preds_val = model.predict([x_val_f], batch_size=512)

        for idx, p in enumerate(preds_val):
            unicorn = unicornify(TRAIN_DF.loc[val_idx[idx]])
            if unicorn is not None:
                preds_val[idx][0] = unicorn
        
        
        preds_test = model.predict(X_test)
        preds.append(preds_test)
        
        for idx, p in enumerate(preds_test):
            unicorn = unicornify(TEST_DF.loc[idx])
            if unicorn is not None:
                preds_test[idx][0] = unicorn
        
        historys.append(history)
        
        score = roc_auc_score(y_val_f, preds_val)
        aucs += score
    
        preds_val = preds_val > threshold
        for idx in range(len(x_val_f)):
            if preds_val[0][0] != y_val_f[idx]:
                missfits.append(TRAIN_DF.values[train_idx[idx]])
    
        fold+=1
    
        print('Fold {}, AUC = {}'.format(fold, score))
    
    preds = np.asarray(preds)[...,0]
    preds = np.mean(preds, axis=0)
    print("Cross Validation AUC = {}".format(aucs/n_splits))
    
    return (preds, historys, missfits)


In [373]:
preds, historys, missfits = train_model(X_train, y_train, X_test, n_splits=10, verbose=0)

Fold 1, AUC = 0.997925925925926
Fold 2, AUC = 1.0
Fold 3, AUC = 0.9994388327721662
Fold 4, AUC = 1.0
Fold 5, AUC = 0.9994019138755981
Fold 6, AUC = 1.0
Fold 7, AUC = 1.0
Fold 8, AUC = 1.0
Fold 9, AUC = 1.0
Fold 10, AUC = 0.9982905982905983
Cross Validation AUC = 0.9995057270864288


In [374]:
sub_df = pd.DataFrame({'vid_id':TEST_DF['vid_id'].values,'is_turkey':preds})
columns_titles = ['vid_id', 'is_turkey']
sub_df = sub_df.reindex(columns=columns_titles)
sub_df.to_csv('submission.csv', index=False)
sub_df.head()

,vid_id,is_turkey
0,pyKh38FXD3E,0.0
1,THhP1idrWXA,0.0
2,jsw3T6GY2Nw,0.0
3,nFkXTMHcjMU,0.0
4,Au8g9kAlrLQ,1.0


In [375]:
! head submission.csv

vid_id,is_turkey
pyKh38FXD3E,0.0
THhP1idrWXA,0.0
jsw3T6GY2Nw,0.0
nFkXTMHcjMU,0.0
Au8g9kAlrLQ,1.0
tgk3DUpRI-w,0.0006214293
BczNwyCGlH4,0.00010217001
zB5eXpmJDTM,9.269991e-11
099z8epFcfc,1.0


In [379]:
! wc -l submission.csv

1197 submission.csv


In [382]:
! kaggle competitions submit -c dont-call-me-turkey -f submission.csv -m "🤞"

100%|██████████████████████████████████████| 22.9k/22.9k [00:01<00:00, 13.1kB/s]
Successfully submitted to Don't call me turkey!